In [55]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.wrappers import TimeDistributed
from keras.layers import LSTM
import numpy as np
import random
import sys
import collections
from music_csv_reader import CSV_Manager


In [56]:
csv_data_manager = CSV_Manager(filename = 'alb1.csv')
csv_data_manager.process_file()
notes = csv_data_manager.music_sequence

In [57]:
maxlen = 20
vec_len = csv_data_manager.vec_length
print(vec_len)


note_sequences = []
next_notes = []
for i in range(0, len(notes) - maxlen, 1):
    note_sequences.append(notes[i: i + maxlen])
    next_notes.append(notes[i])
    
print('nb sequences:', len(note_sequences[0]))
print('nb sequences:', len(next_notes[0]))


note_sequences = np.array(note_sequences)
next_notes = np.array(next_notes)

print(note_sequences.shape)
print(next_notes.shape)


271
nb sequences: 20
nb sequences: 271
(2932, 20, 271)
(2932, 271)


In [58]:
print('Build model...')
model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, vec_len),return_sequences=True))
# model.add(TimeDistributed(Dense(vec_len,activation='sigmoid')))
model.add(LSTM(128, input_shape=(maxlen, vec_len)))
model.add(Dense(vec_len,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 128)               204800    
_________________________________________________________________
dense_11 (Dense)             (None, 271)               34959     
Total params: 239,759.0
Trainable params: 239,759
Non-trainable params: 0.0
_________________________________________________________________
None


In [ ]:
model.fit(note_sequences, next_notes, batch_size=5, epochs=40,verbose=1)

Epoch 1/40
1775/2932 [=================>............] - ETA: 9s - loss: 0.0690

In [20]:
note_sequences

i = np.random.randint(len(note_sequences)-maxlen, size=1)[0]
print(i)
test_x = note_sequences[i]
print(test_x.shape)

test_x=test_x.reshape(1,maxlen,vec_len)


result=test_x.astype(int)

print(result.shape)

for x in range(30):
    pred = model.predict(test_x, verbose=0)
    pred[pred > 0.05] = 1
    pred[pred <= 0.05] = 0
    test_x = pred.astype(int)
    result=np.vstack((result,test_x))

result=result.reshape(-1,vec_len)
print(result.shape)

with open("alb2_predict.csv", "w") as csv_file:
    result = csv_data_manager.form_csv(result)
    csv_file.write(result)
    



453
(20, 271)
(1, 20, 271)
(620, 271)
